In [4]:
import pandas as pd
import numpy as np
from sklearn.svm import SVR
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt

In [5]:
CITY = "milano"
data_dir = "../../preprocessed/" 
features_dir = data_dir + "features/" + CITY + "/"
labels_dir = data_dir + "labels/" + CITY + "/"


PCA_components = 32

standardize_features = True

feature_size = 2048

network_type='VGG'

In [6]:
label_columns = ["hType_mix", "num_intersect", "bld_avg_age", "emp_rat_num",\
				"LUM5_single",	"RNR_nres", "mdist_smallparks", "nig_rat_daily",\
				"nig_rat_daily3", "mdist_nres_daily", "num_community_places", \
				"num_community_places_poi", "avg_block_area", "sphi", \
				"enterprises_empl_size", "pop_rat_num",  \
				"emp_rat_pop", "bld_rat_area", "den_nres_daily",\
				"mdist_parks", "den_nres_non-daily", "mdist_railways",\
				"mdist_highways", "mdist_water"]

In [7]:
def get_normalized_labels_features(network_type='VGG'):
    if network_type == 'ResnNet':
        return pd.read_csv(features_dir + "Resnet50/df_ResNet50_feat8192_pca"+\
            str(PCA_components) +"_labels_features.csv")
    else:
        return pd.read_csv(features_dir+"VGG16/df_VGG16_feat2048_pca"+\
                           str(PCA_components) + "_labels_features.csv")

In [8]:
data = get_normalized_labels_features()

In [9]:
data.head()

,index,imageName,PCA0,PCA1,PCA2,PCA3,PCA4,PCA5,PCA6,PCA7,...,label_pop_rat_num,label_emp_rat_num,label_emp_rat_pop,label_bld_rat_area,label_den_nres_daily,label_mdist_parks,label_den_nres_non-daily,label_mdist_railways,label_mdist_highways,label_mdist_water
0,0,S2B_MSIL2A_20181024T102059_N0209_R065_T32TNR_2...,-0.513386,-0.352732,0.716721,0.755410,0.647604,-0.517665,0.458571,1.024984,...,1.0,0.0,2.0,1.0,1.0,2.0,1.0,2.0,2.0,0.0
1,1,S2B_MSIL2A_20181024T102059_N0209_R065_T32TNR_2...,-0.773019,-0.161784,-0.752245,-0.568582,-0.140501,0.249631,-0.200838,-0.406981,...,1.0,2.0,0.0,2.0,1.0,1.0,1.0,1.0,2.0,0.0
2,2,S2B_MSIL2A_20181024T102059_N0209_R065_T32TNR_2...,-0.305169,0.001131,0.561944,2.024687,0.274702,0.433355,-0.114322,-0.317631,...,0.0,0.0,2.0,0.0,0.0,2.0,1.0,1.0,2.0,1.0
3,3,S2B_MSIL2A_20181024T102059_N0209_R065_T32TNR_2...,-0.011016,-0.778505,1.975514,-1.786100,-0.994235,1.492313,-0.930522,1.202427,...,1.0,0.0,2.0,1.0,1.0,2.0,1.0,2.0,2.0,0.0
4,4,S2B_MSIL2A_20181024T102059_N0209_R065_T32TNR_2...,0.414278,-0.994471,1.943810,-0.590098,-0.266740,0.700525,1.007925,-0.608827,...,0.0,1.0,1.0,1.0,1.0,1.0,0.0,2.0,1.0,0.0


In [12]:
def predict_label_i(label="label_hType_mix"):
    
    data2 = data.copy()
    data2 = data2[data2[label] != 1 ]
    
    target = data2[["imageName", label]]
    features = data2[[c for c in data.columns if "PCA" in c]]
    
    X_orig = features.values
    y_orig = target[label].values
    
    high = target[target[label] == 2]
    low = target[target[label] == 0]
    
    rus = RandomUnderSampler(random_state=0)
    X, y = rus.fit_sample(X_orig, y_orig)
    
    high = target[target[label] == 2]
    low = target[target[label] == 0]
    
    clf = SVR(C=1.0, epsilon=0.2, gamma='auto')
    clf.fit(X, y)
    
#     # #############################################################################
#     # Fit regression model
#     svr_rbf = SVR(kernel='rbf', C=100, gamma=0.1, epsilon=.1)
#     svr_lin = SVR(kernel='linear', C=100, gamma='auto')
#     svr_poly = SVR(kernel='poly', C=100, gamma='auto', degree=3, epsilon=.1,
#                coef0=1)    
    
    
#    # #############################################################################
#     # Look at the results
#     lw = 2

#     svrs = [svr_rbf, svr_lin, svr_poly]
#     kernel_label = ['RBF', 'Linear', 'Polynomial']
#     model_color = ['m', 'c', 'g']

#     fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(15, 10), sharey=True)
#     for ix, svr in enumerate(svrs):
# #         axes[ix].plot(X, svr.fit(X, y).predict(X), color=model_color[ix], lw=lw,
# #                       label='{} model'.format(kernel_label[ix]))
#         axes[ix].scatter(X[svr.support_], y[svr.support_], facecolor="none",
#                          edgecolor=model_color[ix], s=50,
#                          label='{} support vectors'.format(kernel_label[ix]))
#         axes[ix].scatter(X[np.setdiff1d(np.arange(len(X)), svr.support_)],
#                          y[np.setdiff1d(np.arange(len(X)), svr.support_)],
#                          facecolor="none", edgecolor="k", s=50,
#                          label='other training data')
#         axes[ix].legend(loc='upper center', bbox_to_anchor=(0.5, 1.1),
#                         ncol=1, fancybox=True, shadow=True)

#     fig.text(0.5, 0.04, 'data', ha='center', va='center')
#     fig.text(0.06, 0.5, 'target', ha='center', va='center', rotation='vertical')
#     fig.suptitle("Support Vector Regression", fontsize=14)
#     plt.show()
    print(clf.score(X, y), "Low", len(low), "High", len(high), "NOW", len(X)/2)

In [13]:
predict_label_i()

0.817631975948936 Low 71 High 152 NOW 71.0


In [14]:
for col in label_columns:
    label = "label_" + col
    print (label)
    predict_label_i(label)

label_hType_mix
0.817631975948936 Low 71 High 152 NOW 71.0
label_num_intersect
0.8464004021190692 Low 181 High 65 NOW 65.0
label_bld_avg_age
0.8326679684302902 Low 160 High 71 NOW 71.0
label_emp_rat_num
0.8831638617986238 Low 161 High 70 NOW 70.0
label_LUM5_single
0.9012458874206342 Low 61 High 165 NOW 61.0
label_RNR_nres
0.8104157333581896 Low 67 High 141 NOW 67.0
label_mdist_smallparks
0.7673176345086301 Low 132 High 87 NOW 87.0
label_nig_rat_daily
0.7382980425323371 Low 110 High 114 NOW 110.0
label_nig_rat_daily3
0.8790682902002809 Low 167 High 70 NOW 70.0
label_mdist_nres_daily
0.8711955592226016 Low 174 High 60 NOW 60.0
label_num_community_places
0.8891057342311814 Low 177 High 61 NOW 61.0
label_num_community_places_poi
0.8202127173738001 Low 142 High 67 NOW 67.0
label_avg_block_area
0.8438525839987778 Low 74 High 171 NOW 74.0
label_sphi
0.7799324503195302 Low 137 High 91 NOW 91.0
label_enterprises_empl_size
0.8013514768157167 Low 85 High 158 NOW 85.0
label_pop_rat_num
0.921290696

In [15]:
def predict_label_i_KFold(label="label_hType_mix"):
    
    kf = KFold(n_splits=3)
    
    
    data2 = data.copy()
    data2 = data2[data2[label] != 1 ]
    
    target = data2[["imageName", label]]
    features = data2[[c for c in data.columns if "PCA" in c]]
    
    X = features.values
    y = target[label].values
    
    rus = RandomUnderSampler(random_state=0)
    X_resampled, y_resampled = rus.fit_sample(X, y)
    
    res = []
    
    kf.get_n_splits(X_resampled)
    for train_index, test_index in kf.split(X_resampled):
        
        X_train, X_test = X_resampled[train_index], X_resampled[test_index]
        y_train, y_test = y_resampled[train_index], y_resampled[test_index]
    
#         clf = SVR(C=1.0, epsilon=0.2, gamma='scale')
#         clf = SVR(kernel='rbf', C=100, gamma=0.1, epsilon=.1)
        clf = SVR(kernel='linear', C=100, gamma='auto')
#         clf = SVR(kernel='poly', C=100, gamma='auto', degree=3, epsilon=.1,coef0=1)   
        clf.fit(X_train, y_train)
        res.append(clf.score(X_test, y_test))
    return np.mean(res), np.std(res)

In [16]:
predict_label_i_KFold(label="label_hType_mix")

(-0.025634734069751437, 0.03625298858927013)

In [17]:
kfold_SCORES = {}
for col in label_columns:
    label = "label_" + col
    kfold_SCORES[label] = predict_label_i_KFold(label)

In [18]:
kfold_SCORES

{'label_hType_mix': (-0.025634734069751437, 0.03625298858927013),
 'label_num_intersect': (0.07185375255245226, 0.1016165513670784),
 'label_bld_avg_age': (-0.15473675756376037, 0.21883082114430755),
 'label_emp_rat_num': (-0.0013555356279404407, 0.001917016869313301),
 'label_LUM5_single': (0.033737681495739816, 0.04771228673429906),
 'label_RNR_nres': (-0.20353805558445207, 0.287846278666581),
 'label_mdist_smallparks': (-0.287939430897892, 0.40720784831778944),
 'label_nig_rat_daily': (-0.2519908489984252, 0.35636887624748353),
 'label_nig_rat_daily3': (-0.09793975496522285, 0.13850772976731585),
 'label_mdist_nres_daily': (-0.042474775744693126, 0.06006840391690079),
 'label_num_community_places': (-0.025101234603690736, 0.03549850640884828),
 'label_num_community_places_poi': (-0.17054393694873435, 0.2411855486134021),
 'label_avg_block_area': (-0.05562742165727964, 0.07866905414757171),
 'label_sphi': (-0.4203600358676726, 0.5944788638037033),
 'label_enterprises_empl_size': (-0.